In [2]:
!pip install transformers datasets evaluate accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00


## Load coherence data

### Load classifier data fromat

In [3]:
"For data in classifier format: one story + label"
import pandas as pd
classifier_csv = '/content/drive/MyDrive/long-narrative-evaluation/synthetic_classifier_coherence.csv'
# Read the CSV file back into a DataFrame
classifier_df = pd.read_csv(classifier_csv)

import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

# Splitting the DataFrame into train, test, and unsupervised datasets
train_df, test_df = train_test_split(classifier_df, test_size=0.2, random_state=42)

# Select only the 'story' column and rename it to 'text'
train_df = train_df[['story', 'label']].rename(columns={'story': 'text'})
test_df = test_df[['story', 'label']].rename(columns={'story': 'text'})

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Create a DatasetDict
classifier_dataset = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

In [4]:
dataset_dict = classifier_dataset

### Load preference format

In [5]:
"For data in preference format: story1 + story2 + preference_label"
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
preference_csv = '/content/drive/MyDrive/long-narrative-evaluation/synthetic_preference_coherence.csv'
# Read the CSV file back into a DataFrame
preference_df = pd.read_csv(preference_csv)

# Splitting the DataFrame into train and test datasets
train_df, test_df = train_test_split(preference_df, test_size=0.2, random_state=42)

# Creating the training example: "story1 [SEP] story2" and the label
train_df['text'] = train_df.apply(lambda x: f"{x['story1']} [SEP] {x['story2']}", axis=1)
train_df = train_df[['text', 'preference']].rename(columns={'preference': 'label'})

test_df['text'] = test_df.apply(lambda x: f"{x['story1']} [SEP] {x['story2']}", axis=1)
test_df = test_df[['text', 'preference']].rename(columns={'preference': 'label'})

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Create a DatasetDict
preference_dataset = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

In [6]:
dataset_dict = preference_dataset

## Load relevance data

### Load classifier data fromat

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

preference_csv = '/content/drive/MyDrive/long-narrative-evaluation/synthetic_classifier_relevance.csv'
# Read the CSV file back into a DataFrame
preference_df = pd.read_csv(preference_csv)

# Splitting the DataFrame into train and test datasets
train_df, test_df = train_test_split(preference_df, test_size=0.2, random_state=42)

# Creating the training example: "story [SEP] premise" and the label
train_df['text'] = train_df.apply(lambda x: f"{x['story']} [SEP] {x['premise']}", axis=1)

test_df['text'] = test_df.apply(lambda x: f"{x['story']} [SEP] {x['premise']}", axis=1)
# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Create a DatasetDict
preference_dataset = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})


In [17]:
dataset_dict = classifier_dataset

### Load preference format

In [14]:
"For data in preference format: story1 + story2 + preference_label"
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
preference_csv = '/content/drive/MyDrive/long-narrative-evaluation/synthetic_preference_coherence.csv'
# Read the CSV file back into a DataFrame
preference_df = pd.read_csv(preference_csv)

# Splitting the DataFrame into train and test datasets
train_df, test_df = train_test_split(preference_df, test_size=0.2, random_state=42)

# Creating the training example: "story1 [SEP] story2" and the label
train_df['text'] = train_df.apply(lambda x: f"{x['story1']} [SEP] {x['story2']}", axis=1)
train_df = train_df[['text', 'preference']].rename(columns={'preference': 'label'})

test_df['text'] = test_df.apply(lambda x: f"{x['story1']} [SEP] {x['story2']}", axis=1)
test_df = test_df[['text', 'preference']].rename(columns={'preference': 'label'})

# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Create a DatasetDict
preference_dataset = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

In [15]:
dataset_dict = preference_dataset

## Main

In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [18]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096")

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_data = dataset_dict.map(preprocess_function, batched=True)

Map:   0%|          | 0/662 [00:00<?, ? examples/s]

TypeError: ignored

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

import evaluate
accuracy = evaluate.load("accuracy")

import numpy as np
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
# for classifier
id2label = {0: "coherent", 1: "incoherent"}
label2id = {"coherent": 0, "incoherent": 1}

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "allenai/longformer-base-4096", num_labels=2, id2label=id2label, label2id=label2id
)

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
training_args = TrainingArguments(
    output_dir="longformer-relevance-synthetic-classifier",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
    gradient_accumulation_steps=4
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.push_to_hub()

In [ ]:
text = "Aimee Kincaid came home to an empty apartment. She felt alone, even though she had spent the day around people, listening to them talk and laugh and complain. She knew how it sounded when a person sat in solitude after a long trip. She would return from a long journey and sit on her bed and watch the light through the blinds or listen to her neighbor’s conversations with his wife on their balcony. Maybe that was why she felt so strange. Silence was all-encompassing. She sat on her couch and listened to the silence, stared at the flickering lights, waiting for them to hypnotize her into a dreamy state of mind. So different, out of place. She  lit candles strategically placed around the room so that shadows could not obscure her vision or movement. She went into the bathroom and turned on the shower and then, without really knowing why she wanted to, she moved through the apartment turning on all the lights. She was very tired but could not sleep. She turned off all the lights except for the bedside lamp and lay there listening to the ticking of the wall clock and to the faint drone of cars passing on the street below. Aimee felt a sudden sense of panic and fear. She had the feeling that someone was watching her, that someone or something was waiting for her to be completely vulnerable before striking out at her. But she found nothing there, only the light from the lamp moving gently against her walls and ceiling. She turned them on in the kitchen, in her bedroom, even in the closet where nothing but old clothes were kept neatly folded. In her apartment. Silence was all-encompassing in her own small corner of the world, and it was suffocating to her. She started to cry. She cried because the silence was terrifying. She cried because it was so loud, so overwhelming. She cried because she felt alone in a way that no one should have to feel alone, her own small safe island surrounded by an ocean of noise and voices that had suddenly turned into nothing more than a silent and lonely roaring emptiness. She felt exhausted and drained, drained of all the energy it took to simply get through her day. She felt lonely, unable to see past the walls that surrounded her, unable to even reach out for something solid with which she could hold on. She cried because she suddenly felt so alone, more alone than even being at work with all those people who were unable to see past her mask, and all she wanted was to be surrounded by voices and noise."

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("lenguist/longformer-coherence-1")
inputs = tokenizer(text, return_tensors="pt")

tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

NameError: ignored

In [ ]:
import torch

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("lenguist/longformer-coherence-1")
inputs = tokenizer(text, return_tensors="pt")

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("lenguist/longformer-coherence-1")
with torch.no_grad():
    logits = model(**inputs).logits

In [ ]:
logits

tensor([[-0.2519,  0.5050]])

In [ ]:
predicted_class_id = logits.argmax().item()

In [ ]:
predicted_class_id

1